# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,57.52,86,93,28.30,SH,1727950909
1,1,adamstown,-25.0660,-130.1015,69.31,74,67,17.22,PN,1727950910
2,2,madang,-5.2246,145.7966,79.38,89,94,3.65,PG,1727950912
3,3,coquimbo,-29.9533,-71.3436,49.73,100,0,3.44,CL,1727950913
4,4,grytviken,-54.2811,-36.5092,24.57,83,56,10.40,GS,1727950914


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# %%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["City", "Country"]
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
vacay_df = city_data_df[
    (city_data_df['Max Temp'] <= 90) & 
    (city_data_df['Humidity'] <= 60) & 
    (city_data_df['Cloudiness'] <= 50) & 
    (city_data_df['Wind Speed'] <= 10)]

# Drop any rows with null values
vacay_df = vacay_df.dropna()

# Display sample data
vacay_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,44,goynuk,40.4003,30.7883,69.91,44,0,4.92,TR,1727950966
58,58,al bawiti,28.3492,28.8659,84.45,35,0,6.80,EG,1727950984
86,86,yinchuan,38.4681,106.2731,66.69,32,4,5.84,CN,1727951017
97,97,paphos,34.7667,32.4167,82.18,47,0,9.22,CY,1727951030
132,132,mount isa,-20.7333,139.5000,74.97,27,6,4.61,AU,1727951076


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacay_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
44,goynuk,TR,40.4003,30.7883,44,
58,al bawiti,EG,28.3492,28.8659,35,
86,yinchuan,CN,38.4681,106.2731,32,
97,paphos,CY,34.7667,32.4167,47,
132,mount isa,AU,-20.7333,139.5000,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
goynuk - nearest hotel: No hotel found
al bawiti - nearest hotel: Old Oasis Hotel
yinchuan - nearest hotel: Yinchuan hotel
paphos - nearest hotel: Mayfair Gardens Hotel
mount isa - nearest hotel: Ibis Styles
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
tamanrasset - nearest hotel: فندق الأمان
meadow lake - nearest hotel: No hotel found
queenstown - nearest hotel: Queens Hotel
chilecito - nearest hotel: Hermosina Hotel
gaoyou - nearest hotel: 北海假日酒店
bom despacho - nearest hotel: Letícia Hotel
yelizovo - nearest hotel: Николина Усадьба
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
kokstad - nearest hotel: Nolangeni Hotel
eldorado at santa fe - nearest hotel: No hotel found
kununurra - nearest hotel: No hotel found
siwa oasis - nearest hotel: Flower of the city hotel
conceicao das alagoas - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
mananara - nearest hotel: Chez Roger
castaic - nearest hotel: Days In

,City,Country,Lat,Lng,Humidity,Hotel Name
44,goynuk,TR,40.4003,30.7883,44,No hotel found
58,al bawiti,EG,28.3492,28.8659,35,Old Oasis Hotel
86,yinchuan,CN,38.4681,106.2731,32,Yinchuan hotel
97,paphos,CY,34.7667,32.4167,47,Mayfair Gardens Hotel
132,mount isa,AU,-20.7333,139.5000,27,Ibis Styles
149,petropavlovsk-kamchatsky,RU,53.0452,158.6483,46,ООО Постоялый двор
161,tamanrasset,DZ,22.7850,5.5228,21,فندق الأمان
162,meadow lake,US,34.8014,-106.5436,34,No hotel found
174,queenstown,ZA,-31.8976,26.8753,25,Queens Hotel
185,chilecito,AR,-29.1619,-67.4974,47,Hermosina Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
# %%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Country", "Hotel Name"])

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)